# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding = "utf-8" )
weather_data = weather_data.rename(columns = {"Longtidue":"Longitude"})
weather_data.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Touros,100,BR,1584146424,100,-5.20,-35.46,75.20,10.60
1,Port Alfred,26,ZA,1584146424,97,-33.59,26.89,69.01,4.00
2,Dabakala,26,CI,1584146424,67,8.37,-4.43,81.41,5.93
3,Jumla,23,NP,1584146424,65,29.27,82.18,25.75,7.40
4,Kapaa,90,US,1584146143,64,22.08,-159.32,77.00,12.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [66]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [67]:

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [36]:
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 90)]
ideal_weather = ideal_weather.loc[(weather_data['Max Temp'] > 70)]
ideal_weather = ideal_weather.loc[(weather_data['Wind Speed'] < 20)]
ideal_weather = ideal_weather.loc[weather_data['Cloudiness'] < 25 ]
 
hotel_df = ideal_weather.dropna().reset_index(drop=True)
print(len(hotel_df))

60


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [37]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Cidreira,0,BR,1584146424,90,-30.18,-50.21,76.08,11.65,
1,Hilton Head,1,US,1584146370,77,32.22,-80.75,73.99,5.82,
2,Kavieng,12,PG,1584146194,77,-2.57,150.80,84.24,12.10,
3,Mar del Plata,0,AR,1584146181,68,-38.00,-57.56,77.00,10.29,
4,Lázaro Cárdenas,0,MX,1584146429,71,17.96,-102.20,79.93,7.23,


In [58]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

print(locations[0])
params = {
        "radius": 5000,
        "key": g_key,
        "keyword" : "hotel"
    }
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(url,params).json()
    
    try:
        print(f"Found the hotel {response['results'][0]['name']}. Adding to list")
        hotel_df.loc[index, "Hotel Name"] =(response['results'][0]['name'])
    except:
        print("Hotel not found")
        hotel_df.loc[index,"Hotel Name"] = "None Found"
hotel_df

[-30.18, -50.21]
Found the hotel Pond Country Club. Adding to list
Found the hotel Days Inn by Wyndham Hilton Head. Adding to list
Found the hotel Nusa Island Retreat. Adding to list
Found the hotel Hotel Dion. Adding to list
Found the hotel City Express Lázaro Cárdenas. Adding to list
Hotel not found
Found the hotel Hotel da Canoa. Adding to list
Hotel not found
Found the hotel Les Embruns Du Baril. Adding to list
Hotel not found
Found the hotel Grand Isla Navidad Resort. Adding to list
Found the hotel Owls Nest Motel. Adding to list
Found the hotel The Prestige Hotel Penang. Adding to list
Found the hotel Hospedaje Claudia. Adding to list
Found the hotel Hotel Poseidon. Adding to list
Found the hotel Sails in the Desert. Adding to list
Found the hotel Sails Port Macquarie by Rydges. Adding to list
Found the hotel Polana Serena Hotel. Adding to list
Found the hotel Silversands Grenada. Adding to list
Found the hotel Banana Resort Sadao. Adding to list
Found the hotel Premier Hotels & 

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Cidreira,0,BR,1584146424,90,-30.18,-50.21,76.08,11.65,Pond Country Club
1,Hilton Head,1,US,1584146370,77,32.22,-80.75,73.99,5.82,Days Inn by Wyndham Hilton Head
2,Kavieng,12,PG,1584146194,77,-2.57,150.80,84.24,12.10,Nusa Island Retreat
3,Mar del Plata,0,AR,1584146181,68,-38.00,-57.56,77.00,10.29,Hotel Dion
4,Lázaro Cárdenas,0,MX,1584146429,71,17.96,-102.20,79.93,7.23,City Express Lázaro Cárdenas
5,Latung,4,PH,1584146429,70,5.50,120.88,81.10,3.56,None Found
6,Arraial do Cabo,0,BR,1584146432,94,-22.97,-42.03,77.00,5.82,Hotel da Canoa
7,Tessalit,0,ML,1584146435,15,20.20,1.01,71.29,11.14,None Found
8,Saint-Philippe,20,RE,1584146435,88,-21.36,55.77,77.00,16.11,Les Embruns Du Baril
9,Tautira,20,PF,1584146210,62,-17.73,-149.15,89.60,8.05,None Found


In [70]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_h = hotel_df[["Latitude", "Longitude"]]

In [71]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations_h, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[hotel_info]

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
    
# Display Map
fig

SyntaxError: invalid syntax (<ipython-input-71-8d40bf49fd04>, line 7)